c:\Users\user\Desktop\data ia projet\ProjetMaladiesCardiaques\DATA


In [41]:
from knn_model import train_knn_model, predict_with_knn
from random_forest_model import train_random_forest_model,predict_with_random_forest
from regression_lineaire_model import train_regression_model,predict_with_regression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [42]:
df=pd.read_csv('donnée_malades_cardiaques.csv',index_col=0)

X = df.drop(columns=["Maladie_cardiaque"])  
Y = df["Maladie_cardiaque"]


In [43]:
# Charger les données
df = pd.read_csv('donnée_malades_cardiaques.csv', index_col=0)
X = df.drop(columns=['Maladie_cardiaque'])  
Y = df['Maladie_cardiaque']

# Entraînement (retourne aussi le scaler)
rl, scaler = train_regression_model(X, Y)  # Utiliser le même scaler

# Prendre un échantillon de test
num_samples = 100
X_sample = X.sample(n=num_samples, random_state=42)
Y_sample = Y.loc[X_sample.index]

# Appliquer la **même normalisation** qu'à l'entraînement
X_sample_scaled = scaler.transform(X_sample)  # Ne pas recréer un scaler !

# Prédictions avec le même modèle et scaler
rl_preds = rl.predict(X_sample_scaled)
rl_probs = rl.predict_proba(X_sample_scaled)
print("Shape de X_train :", X.shape)  # Doit être (N, 16)
print("Shape de X_sample_scaled :", X_sample_scaled.shape)  # Doit être (100, 16)


ValueError: X has 16 features, but LogisticRegression is expecting 10 features as input.

In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
def generate_comparison_table(X, Y, knn, rf, rl, scaler, num_samples=100):
    # Sélectionner aléatoirement des échantillons
    X_sample = X.sample(n=num_samples, random_state=42)
    Y_sample = Y.loc[X_sample.index]
    
    # Vérifiez que X_sample a le bon nombre de caractéristiques avant la normalisation
    print(f"Shape de X_sample avant transformation : {X_sample.shape}")

    # Normalisation des données
    X_scaled = scaler.transform(X_sample)
    
    # Vérifiez la forme de X_scaled après transformation
    print(f"Shape de X_scaled après normalisation : {X_scaled.shape}")
    
    # Si le modèle LogisticRegression (rl) a été formé avec 10 caractéristiques, appliquez une sélection
    # ou transformation ici pour que X_scaled ait également 10 caractéristiques avant de faire la prédiction.
    
    # Prédictions des modèles
    knn_preds = knn.predict(X_scaled)
    knn_probs = knn.predict_proba(X_scaled)
    
    rf_preds = rf.predict(X_scaled)
    rf_probs = rf.predict_proba(X_scaled)
    
    
    rl_preds = rl.predict(X_scaled)
    rl_probs = rl.predict_proba(X_scaled)
    
    # Construction du DataFrame de comparaison
    comparison_df = X_sample.copy()
    comparison_df['Vraie_classe'] = Y_sample.values
    
    comparison_df['KNN_Prediction'] = knn_preds
    comparison_df['KNN_Prob_Sain'] = knn_probs[:, 0] * 100
    comparison_df['KNN_Prob_Malade'] = knn_probs[:, 1] * 100
    
    comparison_df['RF_Prediction'] = rf_preds
    comparison_df['RF_Prob_Sain'] = rf_probs[:, 0] * 100
    comparison_df['RF_Prob_Malade'] = rf_probs[:, 1] * 100
    
    comparison_df['RL_Prediction'] = rl_preds
    comparison_df['RL_Prob_Sain'] = rl_probs[:, 0] * 100
    comparison_df['RL_Prob_Malade'] = rl_probs[:, 1] * 100
    
    return comparison_df


# Charger les données
df = pd.read_csv('donnée_malades_cardiaques.csv', index_col=0)
X = df.drop(columns=['Maladie_cardiaque'])  
Y = df['Maladie_cardiaque']

# Entraînement des modèles
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
knn = train_knn_model(X, Y)
rf = train_random_forest_model(X, Y)
rl, _ = train_regression_model(X, Y)

# Générer la comparaison
tableau_comparaison = generate_comparison_table(X, Y, knn, rf, rl, scaler)

# Afficher les premières lignes
tableau_comparaison.head()


Shape de X_sample avant transformation : (100, 16)
Shape de X_scaled après normalisation : (100, 16)


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 16 features, but LogisticRegression is expecting 10 features as input.